# Real Time Inference Endpoint

## Deploy as Real Time Inference
- Since the model has been trained with good accuracy we can deploy it.
- First we need to write the 4 functions for model inference in a .py script
- Sagemaker API documentation: https://sagemaker.readthedocs.io/en/stable/api/index.html

In [ ]:
%%writefile serve.py

import os
import joblib
import numpy as np

# Load the Model
def model_fn(model_dir):
    pipeline_model = joblib.load(os.path.join(model_dir, model_file_name))
    return pipeline_model

# Load the input data
def input_fn():
    """An input_fn that loads a pickled numpy array"""
    if request_content_type == "application/python-pickle":
        array = np.load(StringIO(request_body))
        return array
    else:
        # Handle other content-types here or raise an Exception
        # if the content type is not supported.
        pass

def predict_fn():
    pass

def output_fn():
    pass

In [ ]:
# Deploy the model
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import get_execution_role

model_artifact = "s3://sagemaker-us-east-1-298138509966/knn-pipeline-run-2022-06-10-10-43-58-988/output/model.tar.gz"

model = SKLearnModel(
    name="heart-disease-knn-pipeline-model",
    framework_version="0.23-1",
    entry_point="serve.py",
    dependencies=["requirements.txt"],
    model_data=model_artifact,
    role=get_execution_role(),
)

predictor = model.deploy(instance_type="ml.m5.large", initial_instance_count=1)

In [ ]:
# Invoke the model
print(predictor.predict(X_test.iloc[0:2]))

In [ ]:
# Delete the endpoint
sm_boto3.delete_endpoint(EndpointName=predictor.endpoint)